In [60]:
import keras
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
keras.__version__

'2.2.4'

# Utilizando convnets con datasets pequeños

Lo primero que tienes que hacer es descargarte el dataset de https://lara.web.cern.ch/lara/train.zip en la terminal de Jupyter y descomprimirlo en la misma carpeta donde se encuentra esta libreta. 

Para descargar otro conjunto de datos desde imagenet se puede descargar la lista con las URL a las imágenes y usar `wget -i`



## Entrenando desde 0 una convNet

Entrenar un modelo de clasificación de imágenes con muy pocos datos es una situación común en la que te encontrarás si acabas dedicándote a hacer Computer Vision en un contexto profesional. 

Tener "pocas" muestras puede significar cualquier cosa entre unos pocos cientos y unas pocas decenas de miles de imágenes. Vamos a ilustrar aqui un ejemplo práctico: vamos a centrarnos en clasificar imágenes como "perros" y "gatos".


## La relevancia del Deep Learning en problemas con pocos datos

Quizás habrás oido muchas veces que el Deep Learning solo funciona cuando se tienen grandes cantidades de datos. Esto en parte es verdad: una de las características del Deep learning es que puede encontrar características interesantes a partir del dataset de entrenamiento por si mismo, y esto a priori es más sencillo cuando se tienen muchos ejemplos disponibles, especialmente en el caso de tener datasets de input con una alta dimensionalidad, como es el caso de las imágenes.

Sin embargo, lo que constituye un dataset "grande" es relativo. Concretamente relativo al tamaño y la profundidad de la red que estamos intentando entrenar. No es posible enrenar una convnet para que resulta un problema completo con solo unas decenas de ejemplos, pero unos pocos cientos puede ser suficiente si el modelo está bien montado (entenderemos que significa bien "montado" a lo largo del curso de Deep Learning).

Como las convnets aprenden características locales, invariantes bajo translaciones, son muy eficientes en cuanto al número de imágenes necesarias para llevar a cabo problemas perceptuales. Así que entrenar una convnet desde 0 con un dataset no muy grande aún nos puede llevar a resultados razonables como veremos aqui.

Pero hay más aún: los modelos de Deep Learning son altamente "reciclables". Uno puede coger, por ejemplo, un problema de clasificación de imagen y un convertidor de voz a texto entrenado sobre un dataset muy grande y luego reutilizarlo para resolver otro problema completamente distinto solo añadiéndole pequeñas modificaciones. Más especificamente, en el caso de Computer Vision, muchos modelos pre-entrenados (normalmente entrenados en el dataset ImageNet) son hechos publicos para que uno pueda descargarlos y utilizarlos para crear potentes modelos de Computer Vision con muy pocos datos. 

Pero aqui nos vamos a limitar a correr un ejemplo sencillito. 


## Los datos

El dataset de gatos vs perros que utilizamos no es un paquete de Keras. Se publicó en Kaggle.com como parte de un problema de Computer Vision a finales de 2013, cuando todavía las ConvNets no eran tan populares. 

Las imágenes son JPGEs de resolución media. Tiene este aspecto:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

No es ninguna sorpresa que la competición de gatos vs perros de Kaggle en 2013 fuera ganada por ConvNets. Los mejores pudieron alcanzar una precisión de hasta 95%. En nuestro ejemplo vamos a quedarnos todavía lejos de esta precisión, pero durante el curso de Deep Learning hemos aprendido como acercarnos a este valor utilizando diversos métodos para mejorar el rendimiento de las redes neuronales. Hay que tener en cuenta que en este ejemplo estamos entrenando aproximadamente sobre solo el 10% de los datos que se utilizaron para el concurso. 
Después de descargar el dataset y descomprimirlo, vamos a crear un nuevo dataset que contiene tres subsets: un set de training que contiene 1000 imágenes de cada clase, un set de validación con 500 imágenes de cada clase, y finalmente un set de test con 500 imágenes de cada clase.

Aqui tenemos unas cuantas líneas de código que nos hacen este reparto automáticamente:



In [3]:
import os, shutil

In [26]:
#wget -i http://image-net.org/api/text/imagenet.synset.geturls?wnid=n02311617 -t 1 -T 3
#wget -i http://image-net.org/api/text/imagenet.synset.geturls?wnid=n02309337 -t 1 -T 3



# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = '/home/jovyan/work/MachineLearning1/train'

# The directory where we will
# store our smaller dataset
base_dir = '/home/jovyan/work/MachineLearning1/gusanos_and_insectos_small'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training cat pictures
train_gusanos_dir = os.path.join(train_dir, 'gusanos')
os.mkdir(train_gusanos_dir)

# Directory with our training dog pictures
train_insectos_dir = os.path.join(train_dir, 'insectos')
os.mkdir(train_insectos_dir)

# Directory with our validation cat pictures
validation_gusanos_dir = os.path.join(validation_dir, 'gusanos')
os.mkdir(validation_gusanos_dir)

# Directory with our validation dog pictures
validation_insectos_dir = os.path.join(validation_dir, 'insectos')
os.mkdir(validation_insectos_dir)

# Directory with our validation cat pictures
test_gusanos_dir = os.path.join(test_dir, 'gusanos')
os.mkdir(test_gusanos_dir)

# Directory with our validation dog pictures
test_insectos_dir = os.path.join(test_dir, 'insectos')
os.mkdir(test_insectos_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['Gusano_{:04d}.jpg'.format(i) for i in range(1,700)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_gusanos_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['Gusano_{:04d}.jpg'.format(i) for i in range(700, 816)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_gusanos_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['Gusano_{:04d}.jpg'.format(i) for i in range(816, 932)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_gusanos_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['Insecto_{:04d}.jpg'.format(i) for i in range(1,700)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_insectos_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['Insecto_{:04d}.jpg'.format(i) for i in range(700, 816)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_insectos_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['Insecto_{:04d}.jpg'.format(i) for i in range(816, 932)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_insectos_dir, fname)
    shutil.copyfile(src, dst)

As a sanity check, let's count how many pictures we have in each training split (train/validation/test):

In [4]:
print('total training cat images:', len(os.listdir(train_gusanos_dir)))

NameError: name 'train_gusanos_dir' is not defined

In [29]:
print('total training dog images:', len(os.listdir(train_insectos_dir)))

total training dog images: 699


In [30]:
print('total validation cat images:', len(os.listdir(validation_gusanos_dir)))

total validation cat images: 116


In [31]:
print('total validation dog images:', len(os.listdir(validation_insectos_dir)))

total validation dog images: 116


In [32]:
print('total test cat images:', len(os.listdir(test_gusanos_dir)))

total test cat images: 116


In [33]:
print('total test dog images:', len(os.listdir(test_insectos_dir)))

total test dog images: 116


"""""""""""""""""""""""""""
Así que efectivamente tenemos 2000 imágenes de entrenamiento, 1000 imágenes de validación y 1000 imágenes de test. En cada uno de estos subsets hay el mismo número de ejemplos de cada clase: esto es lo que se llama un sistema de clasificación binario balanceado, lo cual significa que nuestra precisión de clasificación será una métrica adecuada del éxito de nuestra solución.

## Construyendo nuestra red

En el anterior ejemplo hemos construido una pequeña convnet para resolver el problema de clasificar números escritos a mano usando el dataset MNIST, así que ya estamos familiarizados con la terminología que utiliza keras. Vamos a reutilizar la estructura general que teniamos en el ejemplo anterior: nuestra convnet tendra una pila de capas alternadas de `Conv2D` (con activación `relu` ) y capas  `MaxPooling2D`.

Sin embargo, como estamos tratando con imágenes mayores y un problema más complejo, vamos a crear nuestra red en consecuencia: tendrá una capa más de `Conv2D` + `MaxPooling2D`. Esto sirve para aumentar la capacidad de la red y para reducir aún más el tamaño de los mapas de características, para que no sean tan enormes cuando lleguen al paso de aplanado. Empezamos usando imágenes de input de 150x150 (una elección arbitraria), y acabaremos con mapas de características que tienen un tamaño de 7x7 antes de la capa de aplanamiento.

Es importante tener en cuenta que la profundidad de los mapas de características va creciendo progresivamente según se avanza en la red neuronales ( de 32 a 128) mientras que el tamaño de los mapas de características va disminuyendo (de 148x148 a 7x7). Este patrón lo verás en casi todas las convnets.

Como estamos atacando un problema de clasificación binaria (perro o gato), vamos a acabar la red con una única unidad (una capa densa de tamaño 1) y con una activación sigmoide. Esta unidad codificará la probabilidad de que nuestra red esté mirando a una clase o a otra.

El aspecto final del modelo debe de ser el siguiente:

![modelo_red_animales.png](https://github.com/laramaktub/MachineLearningI/blob/master/modelo_red_animales.png?raw=true)

In [108]:
from keras import layers
from keras import models
from keras import backend as K

K.clear_session()

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3) , activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3) , activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3) , activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3) , activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

Para el paso de compilación utilizaremos el optimizador `RMSprop`(lr=1e-4). Como nuestra red termina con una única unidad sigmoide, vamos a utilizar binary crossentropy como nuestra función de pérdida.

In [109]:
from keras import optimizers

opt = keras.optimizers.rmsprop(lr=0.0001)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])



##  Preprocesado de datos

Las imágenes deben estar formateadas apropiadamente en tensores de flotantes antes de dárselas a la red. Esto es justo lo que vamos a hacer aqui. Antes de preprocesarlas las imágenes son archivos JPEG. Los pasos para poder darlos a nuestra red son a grandes rasgos:

* Leer los archivos con las imágenes.
* Decodificar el contenido del JPEG en una "parrilla" con el RGB de los pixels 
* Convertir esa "parrilla" en tensores de flotantes
* Re-escalar los valores de los pixels (enre 0 y 255) al intervalo [0, 1] ya que las redes neuronales prefieren trabajar con valores pequeños. 

Todo esto puede parecer muy complicado pero gracias a Keras nuestra vida es mucho más fácil y podemos contar con tus herramientas para ocuparse de estos pasos automaticamente. Keras tiene un módulo con herramientas para el tratamiento de imágenes, que se puede encontrar en  `keras.preprocessing.image`. En particular, contiene la clase `ImageDataGenerator` que nos permite automaticamente convertir imágenes que tengamos en el disco duro en tensores pre-procesados. Esto es justamente lo que usaremos a continuación. Para ello podemos utilizar el flow_from_directory para coger las imágenes directamente de las carpetas que generamos previamente. Le damos como entrada las carpetas donde están las imágenes de entrenamiento (o validación), el tamaño de las imágenes (target_size), tamaño del batch que vamos a usar (vamos a empezar por 20) y como solo hay dos categorías, le decimos que vamos a usar binary_crossentropy (class_mode). Al correr estos comandos obtendremos lo siguiente el número total de imágenes y cuantas clases hay.

In [110]:
#ImageDataGenerator: genera dataset de test, train y validation
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(directory='Documents/master/Machine Learning I/ImageClassification/gusanos_and_insectos_small/train',target_size=(150,150), class_mode="binary", batch_size=16)
validation_generator = validation_datagen.flow_from_directory(directory='Documents/master/Machine Learning I/ImageClassification/gusanos_and_insectos_small/validation',target_size=(150,150), class_mode="binary", batch_size=16)

Found 1198 images belonging to 2 classes.
Found 183 images belonging to 2 classes.


Vamos a echar un vistazo a uno de estos generadores: nos lleva a un batch de 150x150 imágenes RGB (dimensiones `(20, 150, 150, 3)`) y etiquetas binarias (dimensión `(20,)`). 20 es el número de ejemplos en cada batch (lo que llamamos el tamaño del batch). El generador genera estos batches de manera indefinida: corre un bucle sin cesar por todas las imágenes que tengamos en la carpeta. Por eso tenemos que escribir `break` para romper el bucle en algún momento.


In [111]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (16, 150, 150, 3)
labels batch shape: (16,)


Ahora vamos a hacer el fit. En este caso, como lo que tenemos es un generator, utilizamos fit_generator. Vamos a correr 30 épocas y a utilizar el dataset de validación.

In [112]:
history = model.fit_generator(train_generator,samples_per_epoch=1000,nb_epoch=30,validation_data=validation_generator,nb_val_samples=100)
#pintar el accuracy

C:\Users\Carlos\Miniconda3\envs\datascience\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
C:\Users\Carlos\Miniconda3\envs\datascience\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=62, epochs=30, validation_steps=100)`
  """Entry point for launching an IPython kernel.


Epoch 1/30
62/62 [==============================] - 76s 1s/step - loss: 0.6941 - acc: 0.5292 - val_loss: 0.6965 - val_acc: 0.5151
Epoch 2/30
62/62 [==============================] - 74s 1s/step - loss: 0.6846 - acc: 0.5588 - val_loss: 0.6855 - val_acc: 0.5746
Epoch 3/30
62/62 [==============================] - 81s 1s/step - loss: 0.6613 - acc: 0.6192 - val_loss: 0.7975 - val_acc: 0.4852
Epoch 4/30
62/62 [==============================] - 82s 1s/step - loss: 0.6441 - acc: 0.6273 - val_loss: 0.6812 - val_acc: 0.5838
Epoch 5/30
62/62 [==============================] - 77s 1s/step - loss: 0.6367 - acc: 0.6352 - val_loss: 0.6779 - val_acc: 0.5805
Epoch 6/30
62/62 [==============================] - 75s 1s/step - loss: 0.6098 - acc: 0.6663 - val_loss: 0.6580 - val_acc: 0.6708
Epoch 7/30
62/62 [==============================] - 75s 1s/step - loss: 0.5963 - acc: 0.6763 - val_loss: 0.6526 - val_acc: 0.6217
Epoch 8/30
62/62 [==============================] - 75s 1s/step - loss: 0.5485 - acc: 0.72

Es una buena idea guardar el modelo después de entrenar:

In [130]:
from keras.models import load_model

model.save('Documents/master/Machine Learning I/ImageClassification/my_model.h5')  # creates a HDF5 file 'my_model.h5'

Ahora evalua el modelo en el dataset de test.

In [125]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(directory='Documents/master/Machine Learning I/ImageClassification/gusanos_and_insectos_small/test',target_size=(150,150), class_mode="binary", batch_size=16)


model.evaluate_generator(test_generator,steps=182,verbose=1)

Found 182 images belonging to 2 classes.
182/182 [==============================] - 48s 264ms/step


[1.2067745469084346, 0.6144098481197101]

Prueba a optimizar la red. Para ello puedes utilizar las herramientas utilizadas en clase. Puedes probar a optimizar en términos de velocidad y de accuracy. Comenta los resultados.